In [30]:
# from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from keras.models import Sequential
from keras.models import Model
from keras.callbacks import EarlyStopping, ModelCheckpoint

import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Activation, Dense, Flatten, BatchNormalization, Conv2D, MaxPool2D,Dropout,GlobalAveragePooling2D
from keras.metrics import categorical_crossentropy
from keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import confusion_matrix
import itertools
import os
import shutil
import random
import glob
import matplotlib.pyplot as plt
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
# import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.image import imread
import pathlib
from sklearn.metrics import classification_report, confusion_matrix

# %matplotlib inline

In [ ]:
# Unzip training data
# from zipfile import ZipFile
# file_name = "/content/drive/MyDrive/tasks/traffic_signal/archive.zip"
# with ZipFile(file_name, 'r') as zip:
#   zip.extractall('/content/drive/MyDrive/tasks/traffic_signal/archive')
#   print('done')

In [31]:
! pip install split-folders


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import splitfolders
# Split with a ratio.
# To only split into training and validation set, set a tuple to `ratio`, i.e, `(.8, .2)`.
splitfolders.ratio("/content/drive/MyDrive/tasks/traffic_signal/data/archive/traffic_Data/DATA", output="/content/drive/MyDrive/tasks/traffic_signal/data/split_data",
    # seed=1337, 
    ratio=(.7, .15,.15), group_prefix=None, move=False) # default values

Copying files: 7 files [00:02,  4.81 files/s]

KeyboardInterrupt: ignored

In [ ]:
train_dir ='/content/drive/MyDrive/tasks/traffic_signal/data/split_data/train'
val_dir = '/content/drive/MyDrive/tasks/traffic_signal/data/split_data/val'
test_dir ='/content/drive/MyDrive/tasks/traffic_signal/data/split_data/test'
inf_dir='/content/drive/MyDrive/tasks/traffic_signal/data/archive/traffic_Data/TEST'

In [ ]:
img_width=331
img_height=331
batch_size=128

In [ ]:

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   rotation_range=30,
                                   zoom_range=0.4,
                                   horizontal_flip=True)

train_generator = train_datagen.flow_from_directory(train_dir,
                                                    batch_size=batch_size,
                                                    subset="training",
                                                    class_mode='sparse',
                                                    target_size=(img_height, img_width))

Found 2896 images belonging to 58 classes.


In [ ]:

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   rotation_range=30,
                                   zoom_range=0.4,
                                   horizontal_flip=True)

train_generator = train_datagen.flow_from_directory(train_dir,
                                                    batch_size=batch_size,
                                                    subset="training",
                                                    class_mode='sparse',
                                                    target_size=(img_height, img_width))

Found 2896 images belonging to 58 classes.


In [ ]:
val_datagen = ImageDataGenerator(rescale = 1./255)

val_generator = val_datagen.flow_from_directory(val_dir,
                                                 batch_size=batch_size,
                                                 class_mode='sparse',
                                                 subset="training",
                                                 target_size=(img_height, img_width))

Found 599 images belonging to 58 classes.


In [ ]:
from keras.applications import NASNetLarge

In [ ]:
base = NASNetLarge(input_shape=(331,331,3),
                  include_top= False,
                  weights='imagenet',
                   )



In [ ]:
base.trainable = False # here we are freezing the waits

In [ ]:
inputs = keras.Input(shape=(331,331,3))
# We make sure that the base_model is running in inference mode here,
# by passing `training=False`. This is important for fine-tuning, as you will
# learn in a few paragraphs.
x = base(inputs, training=False)
# Convert features of shape `base_model.output_shape[1:]` to vectors
x = keras.layers.GlobalAveragePooling2D()(x)
# A Dense classifier with a single unit (binary classification)
outputs = keras.layers.Dense(58)(x)
model = keras.Model(inputs, outputs)

In [ ]:
# model = Sequential()

# model.add(base)
# model.add(Flatten())
# model.add(Dense(256,activation='relu'))
# model.add(Dense(58,activation='softmax'))

In [ ]:
model.compile(optimizer='adam',loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),metrics=['accuracy'])

In [ ]:

model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 331, 331, 3)]     0         
                                                                 
 NASNet (Functional)         (None, 11, 11, 4032)      84916818  
                                                                 
 global_average_pooling2d (G  (None, 4032)             0         
 lobalAveragePooling2D)                                          
                                                                 
 dense (Dense)               (None, 58)                233914    
                                                                 
Total params: 85,150,732
Trainable params: 233,914
Non-trainable params: 84,916,818
_________________________________________________________________


In [ ]:
model.fit_generator(train_generator, epochs=20, validation_data=val_generator)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  """Entry point for launching an IPython kernel.


Epoch 1/20
11/23 [=============>................] - ETA: 49s - loss: 3.3037 - accuracy: 0.1890

UnknownError: ignored

In [ ]:
model.save('/content/drive/MyDrive/tasks/traffic_signal/nesnetlarge.h5')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
model = tf.keras.models.load_model("/content/drive/MyDrive/tasks/traffic_signal/nesnetlarge.h5")

Copying files: 7 files [00:41,  5.97s/ files]


In [ ]:

test_datagen = ImageDataGenerator(rescale = 1./255,
                                   rotation_range=30,
                                   zoom_range=0.4,
                                   horizontal_flip=True)

test_generator = test_datagen.flow_from_directory(test_dir,
                                                    batch_size=batch_size,
                                                    subset="training",
                                                    class_mode='sparse',
                                                    target_size=(img_height, img_width))

Found 675 images belonging to 58 classes.


In [ ]:
model.compile(optimizer='adam',loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),metrics=['accuracy'])

In [ ]:
model.fit_generator(train_generator, epochs=10, validation_data=val_generator)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  """Entry point for launching an IPython kernel.


Epoch 1/10
18/23 [======================>.......] - ETA: 51s - loss: 0.2628 - accuracy: 0.9424 

UnknownError: ignored

In [ ]:
model.evaluate(test_generator,batch_size=20)

In [ ]:
classes = print(glob.glob("/content/drive/MyDrive/tasks/traffic_signal/data/split_data/train/*"))

['/content/drive/MyDrive/tasks/traffic_signal/data/split_data/train/7', '/content/drive/MyDrive/tasks/traffic_signal/data/split_data/train/0', '/content/drive/MyDrive/tasks/traffic_signal/data/split_data/train/1', '/content/drive/MyDrive/tasks/traffic_signal/data/split_data/train/10', '/content/drive/MyDrive/tasks/traffic_signal/data/split_data/train/11', '/content/drive/MyDrive/tasks/traffic_signal/data/split_data/train/12', '/content/drive/MyDrive/tasks/traffic_signal/data/split_data/train/13', '/content/drive/MyDrive/tasks/traffic_signal/data/split_data/train/14', '/content/drive/MyDrive/tasks/traffic_signal/data/split_data/train/15', '/content/drive/MyDrive/tasks/traffic_signal/data/split_data/train/16', '/content/drive/MyDrive/tasks/traffic_signal/data/split_data/train/17', '/content/drive/MyDrive/tasks/traffic_signal/data/split_data/train/18', '/content/drive/MyDrive/tasks/traffic_signal/data/split_data/train/19', '/content/drive/MyDrive/tasks/traffic_signal/data/split_data/train

In [ ]:
print(test_generator.class_indices)

{'0': 0, '1': 1, '10': 2, '11': 3, '12': 4, '13': 5, '14': 6, '15': 7, '16': 8, '17': 9, '18': 10, '19': 11, '2': 12, '20': 13, '21': 14, '22': 15, '23': 16, '24': 17, '25': 18, '26': 19, '27': 20, '28': 21, '29': 22, '3': 23, '30': 24, '31': 25, '32': 26, '33': 27, '34': 28, '35': 29, '36': 30, '37': 31, '38': 32, '39': 33, '4': 34, '40': 35, '41': 36, '42': 37, '43': 38, '44': 39, '45': 40, '46': 41, '47': 42, '48': 43, '49': 44, '5': 45, '50': 46, '51': 47, '52': 48, '53': 49, '54': 50, '55': 51, '56': 52, '57': 53, '6': 54, '7': 55, '8': 56, '9': 57}


In [ ]:
class_ = []
for i in test_generator.class_indices.keys():
  class_.append(int(i))

In [ ]:
class_

[0,
 1,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 2,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 3,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 4,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 5,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 6,
 7,
 8,
 9]